In [1]:
import sys
sys.path.insert(0,'../..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

def exponential_decay(x, N0=1, lambda_=5):
    return N0*np.exp(-lambda_*x)

def exponential_penalty(x, step_penalty=-1, lambda_=5):
    return step_penalty * (1 - exponential_decay(x, lambda_=lambda_))

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        reward = 0
        distance_normalized = self.env.get_target_distance(normalized=True)
        if distance_normalized < 0.1:
            reward += 10000
        else:
            step_base_penalty = -1
            reward = exponential_penalty(x=distance_normalized, step_penalty=step_base_penalty, lambda_=3)
            # if self.env.state.action_denied is True:
                # reward -= 10
            if self.env.state.touching:
                reward -= 500
        return reward

    def check_done(self):
        if self.env.iterations % 500 == 0:
            print("max iterations")
            return True
        if self.env.total_reward < -1000:
            print("reward boundary")
            return True
        if self.env.get_target_distance(False) < 0.1:
            print("target reached")
            return True
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 2
config.num_obstacles = 0
config.sim_mode = 1
config.sim_step_every_x = 50
action_space = wg.WbtActDiscrete(directions=5, speeds=5, dspeed=0.1, dhead=0.1)

In [4]:
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
                action_class= action_space,
                config=config)

/home/mats/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Accepting on Port:  10201
sending: start env 1


In [5]:
model = PPO1("MlpPolicy", env, tensorboard_log="./wbt_2x2_reach_target_mats/")





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.learn(total_timesteps=500000)


sending: reset
max iterations
sending: reset

max iterations
sending: reset
max iterations
sending: reset
max iterations
sending: reset
reward boundary
sending: reset
target reached
sending: reset
target reached
sending: reset
max iterations
sending: reset
max iterations
sending: reset
max iterations
sending: reset
max iterations
sending: reset
sending: reset
reward boundary
sending: reset
target reached
sending: reset
max iterations
sending: reset
reward boundary
sending: reset
sending: reset
sending: reset
sending: reset
max iterations
sending: reset
target reached
sending: reset
target reached
sending: reset
max iterations
sending: reset
